In [27]:
import sys, os

env_root = '/N/project/baby_vision_curriculum/pythonenvs/hfenv/lib/python3.10/site-packages/'
sys.path.insert(0, env_root)

# SCRIPT_DIR = os.path.realpath(os.path.dirname(inspect.getfile(inspect.currentframe()))) #os.getcwd() #
# print('cwd: ',SCRIPT_DIR)
#os.path.realpath(os.path.dirname(inspect.getfile(inspect.currentframe())))
# util_path = os.path.normpath(os.path.join(SCRIPT_DIR, '..', 'util'))
# sys.path.insert(0, util_path)
os.environ['OPENBLAS_NUM_THREADS'] = '38' #@@@@ to help with the num_workers issue
os.environ['OMP_NUM_THREADS'] = '1'  #10

import numpy as np
import torch, torchvision
# from torchvision import transforms as tr
# torchvision.disable_beta_transforms_warning()
# import torchvision.transforms.v2 as tr
# from torch import nn
# from torch.nn import functional as F
import os
# import random
# import time
from tqdm import tqdm
from pathlib import Path
# import math
import argparse
import pandas as pd
import warnings

import transformers

In [28]:
print(torchvision.__version__)

0.13.0+cu116


In [2]:
import av
from transformers import AutoImageProcessor, VideoMAEForVideoClassification

from huggingface_hub import hf_hub_download

In [3]:
def get_config(image_size, args, num_labels=2):
    arch_kw = args.architecture
    if arch_kw=='small2':
        hidden_size = 768
        intermediate_size = 4*768
        num_attention_heads = 6
        num_hidden_layers = 6
        
        config = transformers.VideoMAEConfig(image_size=image_size, patch_size=16, num_channels=3,
                                             num_frames=16, tubelet_size=2, 
                                             hidden_size=hidden_size, num_hidden_layers=num_hidden_layers, num_attention_heads=num_attention_heads,
                                             intermediate_size=intermediate_size, num_labels=num_labels)
    else:
        raise ValueError
    return config

def get_model(image_size, args):
    config = get_config(image_size, args)
    model = transformers.VideoMAEForPreTraining(config)
    return model

def init_model_from_checkpoint(model, checkpoint_path):
    # caution: model class
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

class Args:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
            


In [4]:
model_root = '/N/project/baby_vision_curriculum/trained_models/generative/v2/s2/'

chpt_name = 'model_g1_seed_1133_other_1345_mask50_small2_pre.g0.pt'
# 'model_g1_seed_1133_other_1345_mask50_small2_pre.g2.pt'

chpt_path = model_root+chpt_name

image_size = 224
architecture='small2'
init_checkpoint_path=chpt_path
args = Args(architecture=architecture, init_checkpoint_path=init_checkpoint_path)

pre_model = get_model(image_size, args)

if args.init_checkpoint_path!='na':
    print('args.init_checkpoint_path:',args.init_checkpoint_path)
    # initialize the model using the checkpoint
    pre_model = init_model_from_checkpoint(pre_model, args.init_checkpoint_path)
        

args.init_checkpoint_path: /N/project/baby_vision_curriculum/trained_models/generative/v2/s2/model_g1_seed_1133_other_1345_mask50_small2_pre.g0.pt


## Prepare the dataset

In [9]:
def _get_transform(image_size):

    mean = [0.5, 0.5, 0.5]#np.mean(mean_all, axis=0) #mean_all[chosen_subj] 
    std = [0.25, 0.25, 0.25] #std_all[chosen_subj] 
    
#     [0.485, 0.456, 0.406]  # IMAGENET_DEFAULT_MEAN
#     [0.229, 0.224, 0.225]  # IMAGENET_DEFAULT_STD

    augs = [tr.Resize(image_size), tr.CenterCrop(image_size), 
            tr.ConvertImageDtype(torch.float32), 
             tr.Normalize(mean,std)]
    return tr.Compose(augs)

In [55]:
def read_video_pyav(container, indices):

    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

In [6]:
# sample 16 frames
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

In [19]:
# video.shape
video = np.moveaxis(video, -1,1)

In [18]:
# np.moveaxis(video, -1,1).shape

(16, 3, 360, 640)

In [10]:
image_size = 224
transform = _get_transform(image_size)

In [93]:
xtt = [transform(torch.from_numpy(frame)).unsqueeze(0) 
       for frame in video]
xtt = torch.concat(xtt, axis=0).unsqueeze(0)
# torch.from_numpy(np.asarray([transform(frame) 
#                              for frame in video])).shape

## Initialize the classifier from the checkpoint

In [7]:
def adapt_videomae(source_model, target_model):
    # load the embeddings
    target_model.videomae.embeddings.load_state_dict(
        source_model.videomae.embeddings.state_dict())
#     load the encoder
    target_model.videomae.encoder.load_state_dict(
        source_model.videomae.encoder.state_dict())
    return target_model

In [60]:
num_labels = 101
config_ds = get_config(image_size, args, num_labels=num_labels)
xmodel = VideoMAEForVideoClassification(config=config_ds)
xmodel = adapt_videomae(pre_model, xmodel)
torch.all(xmodel.videomae.embeddings.patch_embeddings.projection.weight==pre_model.videomae.embeddings.patch_embeddings.projection.weight)
# del pre_model

tensor(True)

In [61]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.classifier.parameters():
            param.requires_grad = True

In [62]:
feature_extracting = True
set_parameter_requires_grad(xmodel, feature_extracting)
## Sanity check
# for param in xmodel.videomae.parameters(): #.classifier.parameters():#
#     print(param.requires_grad)

## Apply the classification model

In [111]:
labels = torch.tensor(([[22]]))
labels.shape

torch.Size([1, 1])

In [112]:
# with torch.no_grad():
labels = torch.tensor((22))
outputs = xmodel(pixel_values=xtt, labels=labels)
logits = outputs.logits

In [113]:
logits.shape

torch.Size([1, 101])

In [102]:
predicted_label = logits.argmax(-1).item()

In [103]:
predicted_label

30

In [106]:
pre_model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}

In [85]:
inputs = image_processor(list(video), return_tensors="pt")

/N/project/baby_vision_curriculum/pythonenvs/hfenv/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:147: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  return torch.tensor(value)


In [88]:
for item in inputs.keys():
    print(item)

pixel_values


In [91]:
inputs['pixel_values'].shape

torch.Size([1, 16, 3, 224, 224])

In [94]:
xtt.shape

torch.Size([1, 16, 3, 224, 224])

## Load UCF101

In [24]:
def transform_vid(video):
    # Used with standard video datasets such as torchvision.UCF101
#     print(vid.shape)
    if video.shape[1]!=3: # Make it TCHW
        video = torch.permute(video, (0,3,1,2))
    image_size = 224
#     vid.p
    transform = _get_transform(image_size)
#     xtt = [transform(torch.from_numpy(frame)).unsqueeze(0) 
    xtt = [transform(frame).unsqueeze(0) 
       for frame in video]
    return torch.concat(xtt, axis=0).unsqueeze(0)

def transform_image(image):
#     Used for standard single image datasets such as torchvision.CIFAR10, torchvision.ImageNet
#     if image.shape[0]!=3:
    image_size=224
    num_frames=16
    transform = _get_transform(image_size)
    return transform(image).unsqueeze(0).repeat(num_frames,1,1,1)

# xtt= torch.randint(0,255, (3, 640, 480))#torch.eye(3).unsqueeze(0)
# ytt = transform_image(xtt)
# ytt.shape

In [149]:
ucf_root='/N/project/baby_vision_curriculum/benchmarks/mainstream/ucf101/UCF-101'
annotation_path = '/N/project/baby_vision_curriculum/benchmarks/mainstream/ucf101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/'
frames_per_clip = 16
step_between_clips = 1
frame_rate=10
train=True
transform = transform_vid
output_format= 'TCHW'
num_workers=40
dataset = torchvision.datasets.UCF101(ucf_root, 
                                      annotation_path,
                                      frames_per_clip,
                                      step_between_clips=step_between_clips,
                                      frame_rate=frame_rate,
                                      fold=1,
                                      train=train,
                                      transform=transform,
                                      output_format=output_format,
                                      num_workers=num_workers)
                                      
                                      

  0%|          | 0/833 [00:00<?, ?it/s]

In [150]:
video, _, label = dataset[0]

In [151]:
video.shape

torch.Size([1, 16, 3, 224, 224])

In [134]:
audio.shape

torch.Size([2, 43776])

In [136]:
label

0

### Temp: compute the number of frames in g3

In [118]:
from pathlib import Path
def get_fpathlist(vid_root, subjdir, ds_rate=1):
    """
    # read the image files inside vid_root/subj_dir into a list. 
    # makes sure they're all jpg. also sorts them so that the order of the frames is correct.
    # subjdir = ['008MS']
    """
    
    fpathlist = sorted(list(Path(os.path.join(vid_root, subjdir)).iterdir()), 
                       key=lambda x: x.name)
    fpathlist = [str(fpath) for fpath in fpathlist if fpath.suffix=='.jpg']
    fpathlist = fpathlist[::ds_rate]
    return fpathlist

In [115]:
jpg_root='/N/project/infant_image_statistics/preproc_saber/JPG_30fps/'

In [120]:
g3='BR+CW+EA+ED+JB+KI+LS+SB+TR'

In [121]:
g3 = g3.split('+')
subj_dirs = g3
ds_rate=3

In [123]:
gx_fpathlist = []
for i_subj, subjdir in enumerate(tqdm(subj_dirs)):
    gx_fpathlist += get_fpathlist(jpg_root, subjdir, ds_rate=ds_rate)

100%|█████████████████████████████████████████████| 9/9 [00:25<00:00,  2.88s/it]


In [124]:
print(len(gx_fpathlist))

918390
